# IBM - Applied Data Science Capstone 

## Capstone Project - The Battle of Neighbourhoods

Opening a New Shopping Mall in Bengaluru, Karnataka, India

1. Build a dataframe of neighborhoods in Bengaluru  by web scraping the data from Wikipedia page.
2. Get the geographical coordinates of the neighborhoods using Python  Geocoder package.
3. Obtain the venue data for the neighborhoods from Foursquare API.
4. Explore and cluster the neighborhoods.
5. Visualize the data and derive insights.
6. Select the best cluster to open a new shopping mall.

## 1.0 INTRODUCTION

The city of Bengaluru, in Karnataka is called as the Silicon Valley of India. This city is flooded with IT and Tech companies which boosts the GDP per capita of the city marking its place at fifth position in India [1]. Therefore, the buying potential of individuals is growing rapidly in this city.

For individuals, visiting shopping malls are like a one-stop destination for satisfying all their needs such as grocery shopping, dine at restaurants, fashion outlets, play games and watch movies. Retailers and Property developers are taking advantage of this trend to build more shopping malls to balance and meet the demand and supply.

As a result, there are many shopping malls in Bengaluru and many under construction. Yet, the demand of the consumers is not meet. Therefore, property developers are willing to construct more shopping malls in this city as this gives them a constant income. So now the property developers are left with a simple question - "Where should we construct a mall?". The is an important business decision to be taken as the location of the property decides its success or a failure.

## 2.0 METHODOLOGY

For the Capstone project I have analysed and selected the best location for construction of a mall in the Bengaluru city, Karnataka. This being a Data science project the methodology followed is,
1. Understanding the business problem
2. Data collection
3. Data cleaning
4. Modelling 
5. Data Visualization
6. Business insights
7. Data Driven decision

Make API calls to Foursquare passing in the geographical coordinates of the neighbourhoods in a Python loop. Foursquare will return the venue data in JSON format and we will extract the venue name, venue category, venue latitude and longitude. With the data, we can check how many venues were returned for each neighbourhood and examine how many unique categories can be curated from all the returned venues. 
Then, we will analyse each neighbourhood by grouping the rows by neighbourhood and taking the mean of the frequency of occurrence of each venue category. By doing so, we are also preparing the data for use in clustering. Since we are analysing the “Shopping Mall” data, we will filter the “Shopping Mall” as venue category for the neighbourhoods.

Lastly, we will perform clustering on the data by using k-means clustering. K-means clustering algorithm identifies k number of centroids, and then allocates every data point to the nearest cluster, while keeping the centroids as small as possible. It is one of the simplest and popular unsupervised machine learning algorithms and is particularly suited to solve the problem for this project. We will cluster the neighbourhoods into 3 clusters based on their frequency of occurrence for “Shopping Mall”. 
The results will allow us to identify which neighbourhoods have higher concentration of shopping malls while which neighbourhoods have fewer number of shopping malls. Based on the occurrence of shopping malls in different neighbourhoods, it will help us to answer the question as to which neighbourhoods are most suitable to open new shopping malls.



In [1]:
# Library to handle data in a vectorized manner
import numpy as np 

# Library for data analsysis
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Library to handle JSON files
import json 


# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
import geocoder 


# Library to handle requests
import requests

# library to parse HTML and XML documents
from bs4 import BeautifulSoup 

# Tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# Map rendering library
import folium 

print("Libraries imported.")

Libraries imported.


## 3.0 DATA COLLECTION
### The data regarding neighbourhoods in Bengaluru is scrapped from wikipedia


•  Latitude and longitude coordinates of those neighbourhoods. This is required in order to plot the map. 
•  Venue data, particularly data related to shopping malls. We will use this data to perform clustering on the neighbourhoods. 
• The geographical coordinates are collected using Python Geocoder package 
•  The details for the mall location is scrapped from the FourSquare developers website. 
 
The data required for this project is scrapped the the following sources (links), 
1.https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore
2.https://foursquare.com/explore?ll=10.466705%2C77.533295&mode=url&near=Bangalore%2C%20Karn%C4%81taka&q=Shopping%20Mall
  the following attributes are collected from this website,
    2.1 Neighbourhood 
    2.2 Latitude
    2.3 Longitude
    2.4 Venue Name 
    2.5 Venue Latitude 
    2.6 Venue Longitude 
    2.6 Venue Category

We will use web scraping techniques to extract the data from the Wikipedia page, with the help of Python requests and beautifulsoup packages. Then we will get the geographical coordinates of the neighbourhoods using Python Geocoder package which will give us the latitude and longitude coordinates of the neighbourhoods. After that, we will use Foursquare API to get the venue data for those neighbourhoods.


In [3]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []

In [4]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
print(neighborhoodList)

['List of areas in Bangalore Cantonment', 'List of areas in Bengaluru Pete', 'List of neighbourhoods in Bangalore', 'Adugodi', 'Agara, Bangalore', 'Ananthnagar', 'Anjanapura', 'Arekere', 'Austin Town', 'Babusapalya', 'Bagalur, Bangalore Urban', 'Bahubalinagar', 'Banashankari', 'Banaswadi', 'Basavanagudi', 'Basaveshwaranagar', 'Bellandur', 'Bharathnagar', 'Bilekahalli', 'Bommanahalli', 'Bommasandra', 'Brookefield', 'BTM Layout', 'Byatarayanapura', 'Carmelaram', 'Chamrajpet', 'Chandapura', 'Chickpet', 'Chikkalasandra', 'Cooke Town', 'Cox Town, Bangalore', 'CV Raman Nagar', 'Dasarahalli', 'Devarachikkanahalli', 'Dollars Colony', 'Domlur', 'Ejipura', 'Electronic City', 'Fraser Town, Bangalore', 'Gandhi Bazaar', 'Gandhi Nagar, Bangalore', 'Ganganagar, Bangalore', 'Girinagar', 'Gowdanapalya', 'Hanumanthanagar, Bengaluru', 'HBR Layout', 'Hebbagodi', 'Hebbal', 'Hoodi', 'Horamavu', 'HSR Layout', 'Hulimavu', 'Immadihalli', 'Indiranagar', 'Ittamadu', 'J. P. Nagar', 'Jakkur', 'Jalahalli', 'Jayanag

In [5]:
bl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

bl_df

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"
5,Ananthnagar
6,Anjanapura
7,Arekere
8,Austin Town
9,Babusapalya


### Data cleaning

In [7]:
tr=bl_df.drop([0,1,2],inplace=False)
tr

,Neighborhood
3,Adugodi
4,"Agara, Bangalore"
5,Ananthnagar
6,Anjanapura
7,Arekere
8,Austin Town
9,Babusapalya
10,"Bagalur, Bangalore Urban"
11,Bahubalinagar
12,Banashankari


### Define a function to get coordinates

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bengaluru, Karnataka, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in bl_df["Neighborhood"].tolist() ]
coords

[[13.057763510964275, 77.50457482431187],
 [13.057763510964275, 77.50457482431187],
 [12.888130000000046, 77.57735000000008],
 [12.944020000000023, 77.60800000000006],
 [12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.02753000000007, 77.65049000000005],
 [13.07728748449913, 77.65760577036913],
 [13.044710000000066, 77.55008000000004],
 [12.922310000000039, 77.56988000000007],
 [13.019643510687336, 77.65469211693214],
 [12.938980000000072, 77.57137000000006],
 [12.992220000000032, 77.53444000000007],
 [12.927350000000047, 77.67185000000006],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.91488000000004, 77.61004000000008],
 [12.951940000000036, 77.544450000000

### A temporary dataframe to populate the coordinates into Latitude and Longitude

In [11]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

### Merge the coordinates 

In [12]:
tr['Latitude'] = df_coords['Latitude']
tr['Longitude'] = df_coords['Longitude']

### Check the neighborhoods and its coordinates

In [13]:
print(tr.shape)
tr

(128, 3)


,Neighborhood,Latitude,Longitude
3,Adugodi,12.944020,77.608000
4,"Agara, Bangalore",12.842830,77.487590
5,Ananthnagar,12.954080,77.541350
6,Anjanapura,12.858110,77.559090
7,Arekere,12.885680,77.596680
8,Austin Town,12.963480,77.612970
9,Babusapalya,13.027530,77.650490
10,"Bagalur, Bangalore Urban",13.077287,77.657606
11,Bahubalinagar,13.044710,77.550080
12,Banashankari,12.922310,77.569880


In [14]:
# Save the DataFrame as CSV file
tr.to_csv("bl_df.csv", index=False)

### Get the coordinates of Bengaluru, Karnataka, India

In [16]:
address = 'Bengaluru, Karnataka, India '

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru, Karnataka, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru, Karnataka, India 12.9791198, 77.5912997.


### Map of Bengaluru 

In [17]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, neighborhood in zip(tr['Latitude'], tr['Longitude'], tr['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [18]:
# save the map as HTML file
map_kl.save('map_bl_bang.html')

### Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'RG4H2GV5W3KMNXIQZCWVHSMF2GWLVAVJV2NV0JUEMVM21P4G' 
CLIENT_SECRET = '1QBUYAR5IEGETXA2VCVGD2ZUDVF0H4I2BWW0LUMTSKZYJI2O' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RG4H2GV5W3KMNXIQZCWVHSMF2GWLVAVJV2NV0JUEMVM21P4G
CLIENT_SECRET:1QBUYAR5IEGETXA2VCVGD2ZUDVF0H4I2BWW0LUMTSKZYJI2O


In [20]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(tr['Latitude'], tr['Longitude'], tr['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6702, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94402,77.608,Tommy Hilfiger,12.934552,77.611347,Clothing Store
1,Adugodi,12.94402,77.608,PVR IMAX,12.934595,77.611321,Movie Theater
2,Adugodi,12.94402,77.608,Lot Like Crêpes,12.936421,77.613284,Creperie
3,Adugodi,12.94402,77.608,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
4,Adugodi,12.94402,77.608,Zingron - Naga Kitchen,12.936271,77.615051,Indian Restaurant


In [22]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,27,27,27,27,27,27
Anjanapura,6,6,6,6,6,6
Arekere,78,78,78,78,78,78
Austin Town,100,100,100,100,100,100
BTM Layout,86,86,86,86,86,86
Babusapalya,52,52,52,52,52,52
"Bagalur, Bangalore Urban",12,12,12,12,12,12
Bahubalinagar,24,24,24,24,24,24


In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


### List of unique categories

In [24]:
venues_df['VenueCategory'].unique()[:]

array(['Clothing Store', 'Movie Theater', 'Creperie', 'Burger Joint',
       'Indian Restaurant', 'Multiplex', 'Financial or Legal Service',
       'Café', 'Brewery', 'Breakfast Spot', 'Lounge', 'Bar',
       'Dessert Shop', 'Donut Shop', 'Coffee Shop', 'Mobile Phone Shop',
       'Ice Cream Shop', 'Chinese Restaurant', 'Indian Sweet Shop',
       'Bakery', 'Gym', 'Tea Room', 'Gaming Cafe', 'Shopping Mall',
       "Men's Store", 'Smoke Shop', 'Yoga Studio', 'Italian Restaurant',
       'Arcade', 'Juice Bar', 'Bookstore', 'Kerala Restaurant',
       'Mexican Restaurant', 'Pizza Place', 'Andhra Restaurant',
       'Cosmetics Shop', 'Salon / Barbershop', 'Tibetan Restaurant',
       'Bowling Alley', 'Comfort Food Restaurant', 'Punjabi Restaurant',
       'Electronics Store', 'Eastern European Restaurant',
       'Persian Restaurant', 'Liquor Store', 'Thai Restaurant',
       'Paper / Office Supplies Store', 'Seafood Restaurant',
       'Restaurant', 'Sandwich Place', 'Fried Chicken Joint'

In [25]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### One hot encoding

In [26]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6702, 238)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(127, 238)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.000000,0.00000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.030000,0.01,0.00000,0.000000,0.000000,0.01,0.0000,0.010000,0.00,0.000000,0.00,0.000000

In [28]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

60

In [29]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [30]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Adugodi,0.010000
1,Ananthnagar,0.000000
2,Anjanapura,0.000000
3,Arekere,0.025641
4,Austin Town,0.010000


### k-means clustering

In [31]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1])

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [33]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adugodi,0.010000,1
1,Ananthnagar,0.000000,1
2,Anjanapura,0.000000,1
3,Arekere,0.025641,0
4,Austin Town,0.010000,1


In [34]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(tr.set_index("Neighborhood"), on="Neighborhood")

# check the last columns!
print(kl_merged.shape)
kl_merged.head() 

(127, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,0.010000,1,12.94402,77.60800
1,Ananthnagar,0.000000,1,12.95408,77.54135
2,Anjanapura,0.000000,1,12.85811,77.55909
3,Arekere,0.025641,0,12.88568,77.59668
4,Austin Town,0.010000,1,12.96348,77.61297


In [35]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(127, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
63,Kammanahalli,0.032258,0,13.014300,77.636850
26,Cooke Town,0.027778,0,13.003630,77.622830
27,"Cox Town, Bangalore",0.027778,0,12.995940,77.626440
104,"Shivajinagar, Bangalore",0.020000,0,12.987200,77.604010
39,Gowdanapalya,0.040816,0,12.909680,77.556750
89,Peenya,0.043478,0,13.031850,77.526790
45,Hoodi,0.022222,0,12.991980,77.715060
47,Hulimavu,0.017241,0,12.880640,77.601470
48,Immadihalli,0.026316,0,12.966860,77.761580
87,Padmanabhanagar,0.028571,0,12.918140,77.559400


### Create map

In [36]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
# save the map as HTML file
map_clusters.save('map_clusters_bang.html')

In [38]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
63,Kammanahalli,0.032258,0,13.014300,77.636850
26,Cooke Town,0.027778,0,13.003630,77.622830
27,"Cox Town, Bangalore",0.027778,0,12.995940,77.626440
104,"Shivajinagar, Bangalore",0.020000,0,12.987200,77.604010
39,Gowdanapalya,0.040816,0,12.909680,77.556750
89,Peenya,0.043478,0,13.031850,77.526790
45,Hoodi,0.022222,0,12.991980,77.715060
47,Hulimavu,0.017241,0,12.880640,77.601470
48,Immadihalli,0.026316,0,12.966860,77.761580
87,Padmanabhanagar,0.028571,0,12.918140,77.559400


In [39]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
123,Wilson Garden,0.000000,1,12.949690,77.596790
86,Nayandahalli,0.000000,1,12.942050,77.521000
117,Vasanth Nagar,0.010000,1,12.990730,77.588610
84,Nagarbhavi,0.000000,1,12.956240,77.509360
118,Vidyaranyapura,0.000000,1,13.079270,77.559140
82,"Murphy Town, Bangalore",0.011494,1,12.980030,77.627760
119,"Vijayanagar, Bangalore",0.000000,1,13.076000,77.652380
81,Milk Colony,0.010000,1,13.009660,77.556830
115,Uttarahalli,0.000000,1,12.897570,77.528300
79,Mariyannapalya,0.000000,1,13.049640,77.611820


In [40]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
53,Jalahalli,0.142857,2,13.05450,77.52658
28,Dasarahalli,0.125000,2,13.04224,77.51507
94,Ramachandrapura,0.083333,2,13.06124,77.55112


## RESULTS 

The results from the k-means clustering show that we categorize the neighbourhoods into 3 
<br>Clusters are based on the frequency of occurrence for “Shopping Mall”: 

<br>Cluster 0 [Red]: Areas with moderate number of shopping malls 
<br>Cluster 1 [Purple]: Areas with low number to no existence of shopping malls 
<br>Cluster 2 [Green]: Areas with high concentration of shopping malls

<img src="files/result.jpg">

## Observations

1. From the map it can be seen that the Eastern and the North-Western parts of the city is flooded with malls
2. The count of malls in the southern part of the city is less.
3. This part of Bengaluru being newly developed is the optimal location for building a new Shopping Mall
4. The optimal areas include, 
    <br> 4.1 Electronic city phase - 1 
    <br> 4.2 Electronic city phase - 2
    <br> 4.3 Anekal
    <br> 4.4 Jigani
    <br> 4.5 Surya Nagar phase - 1
    <br> 4.6 Bommasandra
    
<br>This represents a great opportunity and high potential areas to open new shopping malls as there is no competition.